<a href="https://colab.research.google.com/github/vincent4u/vince-file/blob/main/shuffle_Net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
import tensorflow
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Conv2D, DepthwiseConv2D, Dense, Concatenate, Add, ReLU, BatchNormalization, AvgPool2D, MaxPool2D, GlobalAveragePooling2D, Reshape, Permute, Lambda


def stage(x,channels, repetitions, groups):
    x=shufflenet_block(x, channels=channels, strides =2, groups=groups)
    for i in range (repetitions):
        x=shufflenet_block(x, channels=channels, strides =2, groups=groups)
    return x

def shufflenet_block(tensor, channels, strides, groups):
    x=gconv(tensor,channels=channels//4, groups=groups)
    x= BatchNormalization()(x)
    x=ReLU()(x)

    x=channel_shuffle(x,groups)
    x=DepthwiseConv2D(kernel_size=3, strides=strides, padding='same')(x)
    x=BatchNormalization()(x)

    if strides ==2:
        channels=channels-tensor.get_shape().as_list()[-1]
    x=gconv(x, channels=channels, groups=groups)
    x=BatchNormalization()(x)

    if strides==1:
        x=Add()([tensor,x])

    else:
        avg=AvgPool2D(pool_size=3, strides=2, padding='same')(tensor)
        x= Concatenate()([avg,x])

    output=ReLU()(x)
    return output

def gconv(tensor, channels, groups):
    input_ch=tensor.get_shape().as_list()[-1]
    group_ch=input_ch//groups
    output_ch=channels//groups
    groups_list=[]


    for i in range(groups):

        group_tensor=Lambda(lambda x:x[:,:,:,i * group_ch: (i+1)*group_ch])(tensor)
        group_tensor=Conv2D(output_ch, 1)(group_tensor)
        groups_list.append(group_tensor)

    output=Concatenate()(groups_list)
    return output


def channel_shuffle(x, groups):
    _, width, height, channels =x.get_shape().as_list()
    group_ch= channels// groups

    x=Reshape([width, height, group_ch, groups])(x)
    x=Permute ([1, 2, 3, 4])(x)
    x=Reshape([width, height, channels])(x)
    return x


input = Input([224,224, 3])
x= Conv2D(filters=24, kernel_size=3, strides=2, padding='same')(input)
x=BatchNormalization()(x)
x=ReLU()(x)
x=MaxPool2D(pool_size=3, strides=2, padding='same')(x)

repetitions = 3,7,3
initial_channels=384
groups=8

for i, repetitions in enumerate(repetitions):
    channels=initial_channels * (2**1)
    x= stage(x, channels, repetitions, groups)

x=GlobalAvgPool2D()(x)
output =Dense (1000, activation ='softmax')(x)


model=Model(input.output)



ValueError: Invalid value for argument `filters`. Expected a strictly positive value. Received filters=0.

In [42]:
import tensorflow as tf

from tensorflow.keras.layers import Input, Conv2D, DepthwiseConv2D, Dense, Concatenate, Add, ReLU, BatchNormalization, AvgPool2D, MaxPool2D, GlobalAvgPool2D, Reshape, Permute, Lambda

def stage(x, channels, repetitions, groups):
    x = shufflenet_block(x, channels=channels, strides=2, groups=groups)
    for _ in range(repetitions):
        x = shufflenet_block(x, channels=channels, strides=2, groups=groups)
    return x



def shufflenet_block(tensor, channels, strides, groups):
    x = gconv(tensor, channels=channels // 4, groups=groups)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = channel_shuffle(x, groups)
    x = DepthwiseConv2D(kernel_size=3, strides=strides, padding='same')(x)
    x = BatchNormalization()(x)

    if strides == 2:
        channels = channels - tensor.get_shape().as_list()[-1]
    x = gconv(x, channels=channels, groups=groups)
    x = BatchNormalization()(x)

    if strides == 1:
        x = Add()([tensor, x])
    else:
        avg = AvgPool2D(pool_size=3, strides=2, padding='same')(tensor)
        x = Concatenate()([avg, x])

    output = ReLU()(x)
    return output

def gconv(tensor, channels, groups):
    input_ch = tensor.get_shape().as_list()[-1]
    group_ch = input_ch // groups
    output_ch = channels // groups
    groups_list = []

    for i in range(groups):
        group_tensor = Lambda(lambda x: x[:, :, :, i * group_ch: (i + 1) * group_ch])(tensor)
        group_tensor = Conv2D(output_ch, 1)(group_tensor)
        groups_list.append(group_tensor)

    output = Concatenate()(groups_list)
    return output




def channel_shuffle(x, groups):
    _, width, height, channels = x.get_shape().as_list()
    group_ch = channels // groups

    x = Reshape([width, height, group_ch, groups])(x)
    x = Permute([1, 2, 3, 4])(x)
    x = Reshape([width, height, channels])(x)
    return x


input = Input(shape=[224, 224, 3])
x = Conv2D(filters=24, kernel_size=3, strides=2, padding='same')(input)
x = BatchNormalization()(x)
x = ReLU()(x)
x = MaxPool2D(pool_size=3, strides=2, padding='same')(x)

repetitions = 3, 7, 3
initial_channels = 384
groups = 8

for i, repetitions in enumerate(repetitions):
    channels = initial_channels * (2 ** i)
    x = stage(x, channels, repetitions, groups)

x = GlobalAvgPool2D()(x)
output = Dense(1000, activation='softmax')(x)


from tensorflow.keras import Model
model = Model(input, output)
model.summary()




ValueError: Invalid value for argument `filters`. Expected a strictly positive value. Received filters=0.

In [57]:

import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, DepthwiseConv2D, Dense, Concatenate, Add, ReLU, BatchNormalization, AvgPool2D, MaxPool2D, GlobalAveragePooling2D, Reshape, Permute, Lambda

def shufflenet_block(tensor, channels, strides, groups):
    residual_channels = channels - tensor.get_shape().as_list()[-1]  # Calculate residual channels

    x = gconv(tensor, channels=channels // 4, groups=groups)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = channel_shuffle(x, groups)
    x = DepthwiseConv2D(kernel_size=3, strides=strides, padding='same')(x)
    x = BatchNormalization()(x)

    x = gconv(x, channels=residual_channels, groups=groups)  # Use residual_channels here
    x = BatchNormalization()(x)

    if strides == 1:
        x = Add()([tensor, x])
    else:
        avg = AvgPool2D(pool_size=3, strides=2, padding='same')(tensor)
        x = Concatenate()([avg, x])

    output = ReLU()(x)
    return output


def gconv(tensor, channels, groups):
    input_ch = tensor.get_shape().as_list()[-1]
    group_ch = input_ch // groups
    output_ch = channels // groups
    groups_list = []

    for i in range(groups):
        group_tensor = Lambda(lambda x: x[:, :, :, i * group_ch: (i + 1) * group_ch])(tensor)
        group_tensor = Conv2D(output_ch, 1)(group_tensor)
        groups_list.append(group_tensor)

    output = Concatenate()(groups_list)
    return output

def channel_shuffle(x, groups):
    _, width, height, channels = x.get_shape().as_list()
    group_ch = channels // groups

    x = Reshape([width, height, group_ch, groups])(x)
    x = Permute([1, 2, 3, 4])(x)
    x = Reshape([width, height, channels])(x)
    return x

def stage(x, initial_channels, repetitions, groups):
    for i, num_repetitions in enumerate(repetitions):
        channels = initial_channels * (2 ** i)
        x = shufflenet_block(x, channels=channels, strides=2, groups=groups)
        for _ in range(num_repetitions):
            x = shufflenet_block(x, channels=channels, strides=1, groups=groups)
    return x

input_layer = Input(shape=[224, 224, 3])
x = Conv2D(filters=24, kernel_size=3, strides=2, padding='same')(input_layer)
x = BatchNormalization()(x)
x = ReLU()(x)
x = MaxPool2D(pool_size=3, strides=2, padding='same')(x)

repetitions_list = [3, 7, 3]
initial_channels = 384
groups = 8

for i, num_repetitions in enumerate(repetitions_list):
    x = stage(x, initial_channels, num_repetitions, groups)  # Pass num_repetitions directly

x = GlobalAveragePooling2D()(x)
output = Dense(1000, activation='softmax')(x)

model = tf.keras.Model(inputs=input_layer, outputs=output)
model.summary()


TypeError: 'int' object is not iterable

In [64]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, DepthwiseConv2D, Dense, Concatenate, Add, ReLU, BatchNormalization, AvgPool2D, MaxPool2D, GlobalAveragePooling2D, Reshape, Permute, Lambda

def shufflenet_block(tensor, channels, strides, groups):
    residual_channels = channels - tensor.get_shape().as_list()[-1]  # Calculate residual channels

    x = gconv(tensor, channels=channels // 4, groups=groups)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = channel_shuffle(x, groups)
    x = DepthwiseConv2D(kernel_size=3, strides=strides, padding='same')(x)
    x = BatchNormalization()(x)

    x = gconv(x, channels=residual_channels, groups=groups)  # Use residual_channels here
    x = BatchNormalization()(x)

    if strides == 1:
        x = Add()([tensor, x])
    else:
        avg = AvgPool2D(pool_size=3, strides=2, padding='same')(tensor)
        x = Concatenate()([avg, x])

    output = ReLU()(x)
    return output

def gconv(tensor, channels, groups):
    input_ch = tensor.get_shape().as_list()[-1]
    group_ch = input_ch // groups
    output_ch = channels // groups
    groups_list = []

    for i in range(groups):
        group_tensor = Lambda(lambda x: x[:, :, i * group_ch: (i + 1) * group_ch])(tensor)
        group_tensor = Conv2D(output_ch, 1)(group_tensor)
        groups_list.append(group_tensor)

    output = Concatenate()(groups_list)
    return output

def channel_shuffle(x, groups):
    _, width, height, channels = x.get_shape().as_list()
    group_ch = channels // groups

    x = Reshape([width, height, group_ch, groups])(x)
    x = Permute([1, 2, 3, 4])(x)
    x = Reshape([width, height, channels])(x)
    return x

def stage(x, initial_channels, repetitions, groups):
    for i, num_repetitions in enumerate(repetitions):
        channels = initial_channels * (2 ** i)
        x = shufflenet_block(x, channels=channels, strides=2, groups=groups)
        for _ in range(num_repetitions):
            x = shufflenet_block(x, channels=channels, strides=1, groups=groups)
    return x



input_layer = Input(shape=[224, 224, 3])
x = Conv2D(filters=24, kernel_size=3, strides=2, padding='same')(input_layer)
x = BatchNormalization()(x)
x = ReLU()(x)
x = MaxPool2D(pool_size=3, strides=2, padding='same')(x)

repetitions_list = [3, 7, 3]
initial_channels = 384
groups = 8

for i, num_repetitions in enumerate(repetitions_list):
    x = stage(x, initial_channels, num_repetitions, groups)  # Pass num_repetitions directly

x = GlobalAveragePooling2D()(x)
output = Dense(1000, activation='softmax')(x)

model = tf.keras.Model(inputs=input_layer, outputs=output)
model.summary()


TypeError: 'int' object is not iterable